In [1]:
import pandas as pd
import numpy as np
from  datetime import datetime
import time
from itertools import combinations

In [2]:
def get_time_of_day(x): 
    #утро день вечер ночь, можно как то по другому пороги выставить, не разбирался
    t = x.hour
    if  0<= t <= 6:
        return 0
    elif 7<=t<=12:
        return 1
    elif 13<=t<=19:
        return 2
    else:
        return 3

In [3]:
def numdate2date(x):
    # для того чтобы перевести release_date в таймстамп и посчитать любит ли юзер слушать новые или старые треки
    try:
        return time.mktime(datetime.strptime(str(x), '%Y%m%d').timetuple())
    except:
        return 0

In [4]:
def feature2cat(x):
    # самые "умные" пороги для преобразования фичей в категориальные 
    cats = x.describe()
    cat_1 = cats['25%']
    cat_2 = cats['50%']
    cat_3 = cats['75%']
    
    cat_x = np.copy(x)
    cat_x[x<cat_1] = 0
    cat_x[(x>=cat_1)&(x<cat_2)] = 1
    cat_x[(x>=cat_2)&(x<cat_3)] = 2
    cat_x[x>=cat_3] = 3
    return cat_x

In [5]:
def count_statistics(statistic_set, train, features, target):
    # основная функция, которая считает статистики
    # statistic_set -- датафрейм по которому нужно считать их
    # train -- куда мерджить результаты
    # features -- итератор из списков/кортежей фичей, по которым групбаить
    # target -- ну собственно колонка по которой считать статистики
    train = train.copy()
    for f in features:
        f_ = list(f)
        
        counts = statistic_set.groupby(f_).size().reset_index()
        c = '&'.join(f_)+'-'+target+'_size'
        counts.columns = f_+ [c]
        train = pd.merge(train, counts, on=f_, how='left')
        
        counts = statistic_set.groupby(f_)[target].mean().reset_index()
        c = '&'.join(f_)+'-'+target+'_mean'
        counts.columns = f_+ [c]
        train = pd.merge(train, counts, on=f_, how='left')
        
    return train

In [6]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train.sort_values(by='ts_listen', inplace=True) # для того чтобы удалять БУДУЩЕЕ при генерации выборок
test.sort_values(by='ts_listen', inplace=True)

artist_info = pd.read_csv('./api_data/artist_info.csv') 
album_info = pd.read_csv('./api_data/albums_info.csv')
tracks_info = pd.read_csv('./api_data/songs_info.csv')
tracks_info= tracks_info[['media_id', 'disk_number', 'bpm', 'explicit_lyrics', 'gain', 'track_position', 'rank']]

train = pd.merge(train, artist_info, on='artist_id', how='left')
train = pd.merge(train, album_info, on='album_id', how='left')
train = pd.merge(train, tracks_info, on='media_id', how='left')

test = pd.merge(test, artist_info, on='artist_id', how='left')
test = pd.merge(test, album_info, on='album_id', how='left')
test = pd.merge(test, tracks_info, on='media_id', how='left')


del artist_info
del album_info
del tracks_info
train['time_of_day']=train.ts_listen.apply(datetime.utcfromtimestamp).apply(get_time_of_day)
test['time_of_day']=test.ts_listen.apply(datetime.utcfromtimestamp).apply(get_time_of_day)
test['diff_date'] = test.ts_listen - test.release_date.apply(numdate2date)
train['diff_date'] = train.ts_listen - train.release_date.apply(numdate2date)
train_test = pd.concat([train, test])
train_test.fillna(0, inplace=True)

train_test['user_age_cat'] = feature2cat(train_test.user_age)
train_test['diff_date_cat'] = feature2cat(train_test.diff_date)
train_test['media_duration_cat'] = feature2cat(train_test.media_duration)
train_test['nb_fan_cat'] = feature2cat(train_test.nb_fan)
train_test['nb_album_cat'] = feature2cat(train_test.nb_album)
train_test['nb_tracks_cat'] = feature2cat(train_test.nb_tracks)
train_test['fans_cat'] = feature2cat(train_test.fans)
train_test['bpm_cat'] = feature2cat(train_test.bpm)
train_test['track_position_cat'] = feature2cat(train_test.track_position)
train_test['rank_cat'] = feature2cat(train_test['rank'])
train_test['gain_cat'] = feature2cat(train_test['gain'])

train_test.explicit_lyrics = train_test.explicit_lyrics.astype(int)
train_test.radio = train_test.radio.fillna(0).astype(int)

train = train_test.iloc[:train.shape[0]].copy()
test = train_test.iloc[train.shape[0]:].copy()
del train_test

объявим фичи по которым будем считать статистики
ну и собственно их же комбинации (для больших степеней нужно подсократить список конечна)

In [7]:
cat_features = ['user_id', 'media_id','album_genre_id', 'album_id', 'artist_id', 'context_type', 'genre_id', 'listen_type'
               , 'platform_family', 'platform_name', 'time_of_day', 'user_gender', 'user_age_cat', 'diff_date_cat', 
                'media_duration_cat', 'nb_fan_cat', 'nb_album_cat', 'nb_tracks_cat', 'radio'
                , 'bpm_cat', 'track_position_cat', 'rank_cat', 'gain_cat', 'explicit_lyrics'
               ]

посчитаем все для тестовой выборки сначала, так как потом ТРЕЙН ПОРТИТСЯ, а заново считать долга

In [8]:
#make test
test = count_statistics(train, test, combinations(cat_features , 1), 'is_listened')
test = count_statistics(train, test, combinations(cat_features , 2), 'is_listened')
test.fillna(0, inplace=True)

In [9]:
validation = []
num_tracks = 6 # на сколько треков назад делать датасет, хорошо бы настроить тоже
for i in range(num_tracks):
    v = train[train.listen_type==1].groupby(['user_id']).last().reset_index() # GET LAST TRACK
    
    
    tmp = v[['user_id', 'ts_listen']].copy()
    tmp.columns = ['user_id', 'max_time']
    
    
    train = pd.merge(train, tmp, on='user_id', how='left')
    train = train[train.ts_listen < train.max_time] #DELETE "FUTURE"
    train.drop('max_time',axis=1, inplace=True)
    
    # make features
    c = train.columns.difference(['is_listened', 'listen_type', 'ts_listen'])
    print('0')
    v_new = count_statistics(train, v, combinations(cat_features , 1), 'is_listened')
    print('1')
    v_new = count_statistics(train, v_new, combinations(cat_features , 2), 'is_listened')

    v_new['track_num'] = i
    if i == 0:
        validation = v_new.copy()
    else:
        validation = pd.concat([validation, v_new])
    print(i, validation.shape[0])
    
validation.fillna(0, inplace=True)

0
1
0 15971
0
1
1 30779


# validation

In [10]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
X_train_1 = validation[(validation.track_num>0)].copy()
X_test_1  = validation[validation.track_num==0].copy()

In [12]:
c = validation.columns.difference(['name','track_num', 'is_listened', 'title'])

In [13]:
clf1 = xgb.XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.1)
clf1.fit(X_train_1[c], X_train_1['is_listened'])
score_1 = roc_auc_score(X_test_1['is_listened'], clf1.predict_proba(X_test_1[c])[:, 1])
print(score_1)

0.721243202132


In [14]:
clf = xgb.XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.1)

In [15]:
validation = validation[validation.track_num<5]

In [16]:
clf.fit(validation[c], validation['is_listened'])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

# MAKE PREDICTION

In [17]:
test['is_listened'] = clf.predict_proba(test[c])[:, 1]
ans = test[['sample_id', 'is_listened']].copy()
ans.sort_values(by='sample_id', inplace=True)
ans['sample_id'] = ans.sample_id.astype(int)
ans.to_csv('ans.csv', index=False)

In [18]:
ans[:5]

,sample_id,is_listened
774,0,0.974809
7507,1,0.752584
8695,2,0.761466
2948,3,0.688747
15626,4,0.902059
